

# **Carga y preparación inicial de los datos desde Kaggle**

En esta sección se realiza el proceso de **configuración, descarga y carga de los datos** provenientes de la competencia de Kaggle.  
El objetivo es asegurar que la información esté disponible y correctamente organizada antes de iniciar el análisis y el modelado.

Las acciones realizadas son las siguientes:

- **Configuración de credenciales:**  
  Se verifica si existe el archivo de autenticación `kaggle.json`.  
  En caso contrario, se solicita al usuario cargarlo manualmente y luego se configura en la ruta adecuada, asignando los permisos necesarios para permitir el uso seguro de la API.

- **Descarga del dataset:**  
  se descarga el archivo comprimido correspondiente a la competencia especificada, garantizando el acceso a los datos oficiales.

- **Extracción de los archivos:**  
  El archivo descargado se descomprime dentro del directorio `data/`, permitiendo acceder de forma organizada a los diferentes archivos incluidos en el paquete.

- **Carga de los datasets principales:**  
  Se leen los archivos `train.csv`, `test.csv` y `submission\_example.csv`

  El archivo de entrenamiento contiene las variables sobre las cuales se construirá el modelo, el archivo de prueba se utiliza para generar predicciones finales y el archivo de ejemplo sirve como referencia para estructurar el envío a Kaggle.

- **Verificación inicial de los datos:**  
  Se imprimen las dimensiones de los datasets mediante `.shape` y se muestran las primeras filas con `display()`, lo que permite confirmar que los datos se cargaron correctamente y poseen la estructura esperada.





In [2]:
import os
import pandas as pd

# --- cargar kaggle.json si no existe ---
if not os.path.exists("/root/.kaggle/kaggle.json"):
    from google.colab import files
    print("Sube aquí tu archivo kaggle.json")
    uploaded = files.upload()

    if "kaggle.json" not in uploaded:
        raise ValueError("Debes subir un archivo llamado kaggle.json")

    os.makedirs("/root/.kaggle", exist_ok=True)
    with open("/root/.kaggle/kaggle.json", "wb") as f:
        f.write(uploaded["kaggle.json"])

    os.chmod("/root/.kaggle/kaggle.json", 600)
    print("kaggle.json configurado correctamente.")

# --- descargar datos de Kaggle ---
COMP = "udea-ai-4-eng-20252-pruebas-saber-pro-colombia"
!kaggle competitions download -c $COMP --force

# --- extraer ---
os.makedirs("data", exist_ok=True)
!unzip -o {COMP}.zip -d data/

# --- cargar datasets ---
train = pd.read_csv("data/train.csv")
test  = pd.read_csv("data/test.csv")
sample_sub = pd.read_csv("data/submission_example.csv")  # <– corregido

print("Train shape:", train.shape)
print("Test shape:", test.shape)
display(train.head())
display(sample_sub.head())


Sube aquí tu archivo kaggle.json


Saving kaggle.json to kaggle.json
kaggle.json configurado correctamente.
  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.08GB/s]
Archive:  udea-ai-4-eng-20252-pruebas-saber-pro-colombia.zip
  inflating: data/submission_example.csv  
  inflating: data/test.csv           
  inflating: data/train.csv          
Train shape: (692500, 21)
Test shape: (296786, 20)


,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


,ID,RENDIMIENTO_GLOBAL
0,550236,medio-bajo
1,98545,medio-bajo
2,499179,medio-alto
3,782980,alto
4,785185,medio-bajo


### **Exploración inicial**
Se visualiza la estructura general del conjunto de entrenamiento mediante `head()`, `info()` y el conteo de valores nulos.  
Esto permite identificar el tipo de variables y la presencia de datos faltantes.

---

### **Preparación de copias de trabajo**
Se crean copias independientes de los datasets de entrenamiento y prueba (`df` y `df_test`) para trabajar sin modificar los archivos originales.

---

### **Identificación de variables**
- Se detectan las columnas numéricas usando los tipos `int64` y `float64`.
- Se detectan las columnas categóricas tomando las de tipo `object`.
- Se excluye la variable objetivo `RENDIMIENTO_GLOBAL` del listado de categóricas.

---

### **Tratamiento de valores faltantes**
**Para variables numéricas:**  
Se reemplazan los valores nulos por la media calculada en el conjunto de entrenamiento, aplicando el mismo valor al conjunto de prueba.

**Para variables categóricas:**  
Se reemplazan los valores nulos por la moda de cada columna, garantizando consistencia entre train y test.

---

### **Codificación de variables categóricas (One-Hot Encoding)**
Se aplica `pd.get_dummies()` a las columnas categóricas para transformarlas en variables binarias.  
Luego, se alinean `df` y `df_test` para asegurar que ambos tengan exactamente las mismas columnas, rellenando con cero las categorías ausentes en el conjunto de prueba.

---

### **Construcción de matrices de entrenamiento**
- Se separa la variable objetivo `RENDIMIENTO_GLOBAL`.  
- Se construye la matriz de entrada `X` con todas las columnas restantes.

---

### **División en entrenamiento y validación**
Se realiza una partición estratificada del 80% para entrenamiento y 20% para validación, manteniendo la proporción de clases en la variable objetivo.




In [3]:
# Exploración inicial
print(train.head())
print(train.info())
print(train.isnull().sum())

# Copias de trabajo
df = train.copy()
df_test = test.copy()

# Identificar columnas numéricas y categóricas (sin quitar nada todavía)
num_cols = df.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols = df.select_dtypes(include=['object']).columns.tolist()

# Quitar la columna objetivo sólo de las categóricas
target = 'RENDIMIENTO_GLOBAL'
cat_cols.remove(target)

# Relleno de nulos en numéricas → media
for col in num_cols:
    df[col] = df[col].fillna(df[col].mean())
    df_test[col] = df_test[col].fillna(df[col].mean())

# Relleno de nulos en categóricas → moda
for col in cat_cols:
    mode = df[col].mode()[0]
    df[col] = df[col].fillna(mode)
    df_test[col] = df_test[col].fillna(mode)

# One-Hot encoding
df = pd.get_dummies(df, columns=cat_cols)
df_test = pd.get_dummies(df_test, columns=cat_cols)

# Alinear columnas entre train y test
df, df_test = df.align(df_test, join='left', axis=1, fill_value=0)

# Separar X e y
X = df.drop(target, axis=1)
y = df[target]

# División train/valid
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Tamaño de entrenamiento:", X_train.shape)
print("Tamaño de validación:", X_valid.shape)


       ID  PERIODO_ACADEMICO            E_PRGM_ACADEMICO E_PRGM_DEPARTAMENTO  \
0  904256              20212                  ENFERMERIA              BOGOTÁ   
1  645256              20212                     DERECHO           ATLANTICO   
2  308367              20203       MERCADEO Y PUBLICIDAD              BOGOTÁ   
3  470353              20195  ADMINISTRACION DE EMPRESAS           SANTANDER   
4  989032              20212                  PSICOLOGIA           ANTIOQUIA   

                E_VALORMATRICULAUNIVERSIDAD E_HORASSEMANATRABAJA  \
0  Entre 5.5 millones y menos de 7 millones    Menos de 10 horas   
1  Entre 2.5 millones y menos de 4 millones                    0   
2  Entre 2.5 millones y menos de 4 millones      Más de 30 horas   
3  Entre 4 millones y menos de 5.5 millones                    0   
4  Entre 2.5 millones y menos de 4 millones  Entre 21 y 30 horas   

  F_ESTRATOVIVIENDA F_TIENEINTERNET                    F_EDUCACIONPADRE  \
0         Estrato 3              Si

### **Carga de datos**
Se cargan los archivos `train.csv`, `test.csv` y `submission_example.csv`.  
Se separa la variable objetivo `RENDIMIENTO_GLOBAL` del resto de columnas.

---

### **Identificación de tipos de variables**
Se determinan las columnas numéricas y categóricas usando sus tipos de datos.  

---

### **Imputación de valores faltantes**
- En numéricas se reemplazan nulos con la **media** del conjunto de entrenamiento.  
- En categóricas se reemplazan nulos con la **moda** correspondiente.  
Los mismos valores de referencia se aplican al conjunto de prueba.

---

### **Codificación de variables categóricas**
Se aplica **frequency encoding**, reemplazando cada categoría por su frecuencia relativa en el conjunto de entrenamiento.  
Valores no existentes en test se llenan con 0.

---

### **División del conjunto de entrenamiento**
Los datos se dividen en entrenamiento y validación (80/20) usando estratificación según la variable objetivo.

---

### **Entrenamiento del modelo LightGBM**
Se entrena un modelo `LGBMClassifier` con:
- 500 árboles  
- learning rate de 0.05  
- num\_leaves = 50  
- muestreo en filas y columnas del 80%  
Se evalúa usando accuracy sobre el conjunto de validación.

---

### **Entrenamiento final y generación de predicciones**
El modelo se vuelve a entrenar usando todos los datos de entrenamiento.  
Se generan predicciones para el conjunto de prueba.

---

### **Creación del archivo de submission**
Se agregan las predicciones al archivo de ejemplo y se genera  
**`submission_lgbm.csv`**, listo para enviar a Kaggle.


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

# Cargar datos
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")
submission = pd.read_csv("data/submission_example.csv")

# Separar target
target_col = "RENDIMIENTO_GLOBAL"
y = train[target_col]
X = train.drop([target_col], axis=1)

# Identificar columnas
num_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
cat_cols = X.select_dtypes(include=['object']).columns.tolist()

# Imputar valores faltantes
for c in num_cols:
    X[c] = X[c].fillna(X[c].mean())
    test[c] = test[c].fillna(X[c].mean())

for c in cat_cols:
    X[c] = X[c].fillna(X[c].mode()[0])
    test[c] = test[c].fillna(X[c].mode()[0])

# Encoding simple (frequency encoding para reproducibilidad)
for c in cat_cols:
    freq = X[c].value_counts(normalize=True)
    X[c] = X[c].map(freq).astype(float)
    test[c] = test[c].map(freq).fillna(0).astype(float)

# Dividir datos en entrenamiento y validación
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Entrenar LightGBM
model = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=-1,
    num_leaves=50,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)

model.fit(X_train, y_train)

# Validación
val_pred = model.predict(X_valid)
acc = accuracy_score(y_valid, val_pred)
print("Accuracy de validación:", acc)

# Entrenar modelo final con todos los datos
model.fit(X, y)

# Predecir test
test_pred = model.predict(test)

# Preparar submission
submission[target_col] = test_pred
submission.to_csv("submission_lgbm.csv", index=False)
print("Archivo submission_lgbm.csv generado correctamente.")



[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.052834 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1469
[LightGBM] [Info] Number of data points in the train set: 554000, number of used features: 20
[LightGBM] [Info] Start training from score -1.371993
[LightGBM] [Info] Start training from score -1.387089
[LightGBM] [Info] Start training from score -1.395033
[LightGBM] [Info] Start training from score -1.391216
Accuracy de validación: 0.43158844765342963
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.073019 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1471
[LightGBM] [Info] Number of data points in the train set: 692500, number of used features: 20
[LightGBM] [Info] Start training

# **Verificación del archivo de submission generado**

En esta celda se realiza una comprobación rápida del archivo final exportado por el modelo.

- Se valida que el archivo `submission_lgbm.csv` exista en la ruta esperada.  
- Se calcula y muestra su tamaño para confirmar que fue generado correctamente.
- Se cargan las primeras filas del archivo para verificar su estructura.
- Se imprime la información general del dataframe para confirmar que contiene las columnas y tipos de datos adecuados.

Esta revisión asegura que el archivo de submission está completo y listo para enviarse a Kaggle.


In [5]:
from pathlib import Path
import pandas as pd
import os

# Ruta del archivo de submission
submission_path = '/content/submission_lgbm.csv'

# Verifico que el archivo exista
if not Path(submission_path).exists():
    raise FileNotFoundError(f"No se encontró el archivo de submission: {submission_path}")

# Tamaño del archivo en MB
file_size = os.path.getsize(submission_path) / (1024**2)
print(f"Tamaño del archivo: {file_size:.2f} MB")

# Cargo el archivo y muestro las primeras filas
submission_df = pd.read_csv(submission_path)
print("Primeras filas del archivo:")
display(submission_df.head())

# Información general
print("\nInformación del archivo:")
print(submission_df.info())


Tamaño del archivo: 4.03 MB
Primeras filas del archivo:


,ID,RENDIMIENTO_GLOBAL
0,550236,bajo
1,98545,medio-alto
2,499179,alto
3,782980,bajo
4,785185,bajo



Información del archivo:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296786 entries, 0 to 296785
Data columns (total 2 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   ID                  296786 non-null  int64 
 1   RENDIMIENTO_GLOBAL  296786 non-null  object
dtypes: int64(1), object(1)
memory usage: 4.5+ MB
None


In [6]:
# Crear directorio para el token de Kaggle y configurar permisos
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Copiar el archivo de submission a la carpeta de trabajo como 'submission.csv'
!cp /content/submission_lgbm.csv submission.csv

# Subir a la competencia
!kaggle competitions submit -c udea-ai-4-eng-20252-pruebas-saber-pro-colombia -f submission.csv -m "LightGBM - primera versión"


100% 4.03M/4.03M [00:01<00:00, 2.28MB/s]
Successfully submitted to UDEA/ai4eng 20252 - Pruebas Saber Pro Colombia